#### German credit dataset

In [1]:
# Necessary imports
import pandas as pd
import numpy as np
from numpy import genfromtxt
import csv
import seaborn as sns
from scipy import stats
import matplotlib.pyplot as plt
import matplotlib.colors
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from keras.models import load_model
from scipy.io import arff
from io import StringIO
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import time
import random
from sklearn.model_selection import train_test_split 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Lambda, Input, Dense, Layer
#from keras.layers import BatchNormalization,LayerNormalization
from tensorflow.keras.losses import mse, binary_crossentropy,categorical_crossentropy,sparse_categorical_crossentropy
from keras.layers import Activation, Dense, Flatten 
from tensorflow.keras import backend as K
from numpy.random import seed
seed(1)
import tensorflow as tf
tf.random.set_seed(2)
import random
random.seed(3)
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
import scipy
from scipy.stats import multivariate_normal

C:\Users\91948\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
Using TensorFlow backend.


In [2]:
import scipy.io
mat = scipy.io.loadmat('GER_TO_USE (1).mat')

In [3]:
# Train and Test data - 70% Train and 30% Test
x_train = mat['train_x'] 
ytrn = mat['train_y'] 
y_train =  ytrn[:,0]*2+ytrn[:,1]*1

x_test = mat['test_x'] 
ytst = mat['test_y'] 
y_test =  ytst[:,0]*2+ytst[:,1]*1

In [4]:
mmscaler    = MinMaxScaler(feature_range=(-1.5,1.5))
x_train   = mmscaler.fit_transform(x_train)
x_test = mmscaler.transform(x_test)

### VAE

#### VAE code is adopted and modified from the following reference.
#### https://github.com/PacktPublishing/Advanced-Deep-Learning-with-Keras/blob/master/chapter8-vae/vae-mlp-mnist-8.1.1.py 

In [5]:
start_dimension = x_train.shape[1]
# network parameters
input_shape = (start_dimension, )
intermediate_dim = 18
batch_size = 24 
latent_dim = 6
epochs = 1000

In [6]:
def sampling(args):
    z_mean, z_log_var = args
    batch = K.shape(z_mean)[0]
    dim = K.int_shape(z_mean)[1]
    epsilon = K.random_normal(shape=(batch, dim))
    return z_mean + K.exp(0.5 * z_log_var) * epsilon

In [7]:
# Encoder model
inputs = keras.Input(shape=input_shape, name='encoder_input')
x = layers.Dense(intermediate_dim, activation='relu')(inputs)
z_mean = layers.Dense(latent_dim, name='z_mean')(x)
z_log_var = layers.Dense(latent_dim, name='z_log_var')(x)
z = Lambda(sampling,
           output_shape=(latent_dim,), 
           name='z')([z_mean, z_log_var])
encoder = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 24)]         0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 18)           450         encoder_input[0][0]              
__________________________________________________________________________________________________
z_mean (Dense)                  (None, 6)            114         dense[0][0]                      
__________________________________________________________________________________________________
z_log_var (Dense)               (None, 6)            114         dense[0][0]                      
____________________________________________________________________________________________

In [8]:
# Decoder model
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='tanh')(latent_inputs)
outputs = layers.Dense(start_dimension, activation='tanh')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')
decoder.summary()

Model: "decoder"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
z_sampling (InputLayer)      [(None, 6)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 18)                126       
_________________________________________________________________
dense_2 (Dense)              (None, 24)                456       
Total params: 582
Trainable params: 582
Non-trainable params: 0
_________________________________________________________________


In [9]:
# VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

In [10]:
# VAE Train validation split
X_imtrain, X_imval, y_imtrain, y_imval = train_test_split(x_train, x_train, 
                                                    test_size=0.2, random_state=1)

In [11]:
start_time = time.time()

In [12]:
if __name__ == '__main__':
    
    loss = 'mse'
    models = (encoder, decoder)
    
    if loss == 'bce':
        reconstruction_loss = binary_crossentropy(inputs,
                                                  outputs)
    else:
        reconstruction_loss = mse(inputs, outputs)
        
    reconstruction_loss *= start_dimension
    kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
    kl_loss = K.sum(kl_loss, axis=-1)
    kl_loss *= -0.5
    vae_loss = K.mean(reconstruction_loss + kl_loss)
    vae.add_loss(vae_loss)
    opt = keras.optimizers.Adam(lr =0.0001)
    vae.compile(optimizer=opt,)
    vae.summary()
            
    vae.fit(X_imtrain,
            epochs=epochs,
            #verbose = 10,
            batch_size=batch_size,
            validation_data=(X_imval, None))
    

C:\Users\91948\Anaconda3\lib\site-packages\tensorflow\python\keras\optimizer_v2\optimizer_v2.py:375: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Model: "vae_mlp"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
encoder_input (InputLayer)      [(None, 24)]         0                                            
__________________________________________________________________________________________________
encoder (Functional)            [(None, 6), (None, 6 678         encoder_input[0][0]              
__________________________________________________________________________________________________
decoder (Functional)            (None, 24)           582         encoder[0][2]                    
__________________________________________________________________________________________________
dense (Dense)                   (None, 18)           450         encoder_input[0][0]              
____________________________________________________________________________________________

24/24 [==============================] - 0s 6ms/step - loss: 35.4861 - val_loss: 35.1199
Epoch 32/1000
24/24 [==============================] - 0s 6ms/step - loss: 34.9943 - val_loss: 34.5325
Epoch 33/1000
24/24 [==============================] - 0s 6ms/step - loss: 34.6770 - val_loss: 34.8174
Epoch 34/1000
24/24 [==============================] - 0s 6ms/step - loss: 34.5772 - val_loss: 33.9925
Epoch 35/1000
24/24 [==============================] - 0s 5ms/step - loss: 34.2686 - val_loss: 33.4357
Epoch 36/1000
24/24 [==============================] - 0s 5ms/step - loss: 33.8669 - val_loss: 33.1705
Epoch 37/1000
24/24 [==============================] - 0s 6ms/step - loss: 33.5369 - val_loss: 33.7761
Epoch 38/1000
24/24 [==============================] - 0s 7ms/step - loss: 33.7030 - val_loss: 33.4794
Epoch 39/1000
24/24 [==============================] - 0s 7ms/step - loss: 33.0982 - val_loss: 32.4545
Epoch 40/1000
24/24 [==============================] - 0s 7ms/step - loss: 32.7324 - va

24/24 [==============================] - 0s 6ms/step - loss: 26.8112 - val_loss: 26.1616
Epoch 111/1000
24/24 [==============================] - 0s 6ms/step - loss: 27.0105 - val_loss: 26.4347
Epoch 112/1000
24/24 [==============================] - 0s 6ms/step - loss: 27.0571 - val_loss: 26.6558
Epoch 113/1000
24/24 [==============================] - 0s 6ms/step - loss: 26.8316 - val_loss: 26.2011
Epoch 114/1000
24/24 [==============================] - 0s 7ms/step - loss: 27.0393 - val_loss: 25.9489
Epoch 115/1000
24/24 [==============================] - 0s 6ms/step - loss: 26.7952 - val_loss: 25.6937
Epoch 116/1000
24/24 [==============================] - 0s 5ms/step - loss: 26.9088 - val_loss: 26.8582
Epoch 117/1000
24/24 [==============================] - 0s 6ms/step - loss: 26.8390 - val_loss: 27.0484
Epoch 118/1000
24/24 [==============================] - 0s 7ms/step - loss: 26.4142 - val_loss: 26.3298
Epoch 119/1000
24/24 [==============================] - 0s 6ms/step - loss: 26.

Epoch 189/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.2550 - val_loss: 24.7159
Epoch 190/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.3052 - val_loss: 24.6146
Epoch 191/1000
24/24 [==============================] - 0s 4ms/step - loss: 25.0422 - val_loss: 24.7144
Epoch 192/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.1780 - val_loss: 24.7033
Epoch 193/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.3520 - val_loss: 24.4926
Epoch 194/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.1330 - val_loss: 24.5316
Epoch 195/1000
24/24 [==============================] - 0s 4ms/step - loss: 24.9715 - val_loss: 24.6369
Epoch 196/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.0017 - val_loss: 24.7861
Epoch 197/1000
24/24 [==============================] - 0s 5ms/step - loss: 25.1730 - val_loss: 24.7753
Epoch 198/1000
24/24 [==============================] - 0s 6ms/s

...

Epoch 662/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.2968 - val_loss: 19.0874
Epoch 663/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.4208 - val_loss: 19.5318
Epoch 664/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.6695 - val_loss: 19.5528
Epoch 665/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.3482 - val_loss: 19.4279
Epoch 666/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.3602 - val_loss: 19.4783
Epoch 667/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.5046 - val_loss: 19.3401
Epoch 668/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.2472 - val_loss: 19.3921
Epoch 669/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.2296 - val_loss: 19.2346
Epoch 670/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.3262 - val_loss: 19.4708
Epoch 671/1000
24/24 [==============================] - 0s 5ms/s

Epoch 741/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.9354 - val_loss: 19.3481
Epoch 742/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.9970 - val_loss: 19.3322
Epoch 743/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.0451 - val_loss: 19.0425
Epoch 744/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.2087 - val_loss: 19.2701
Epoch 745/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.9922 - val_loss: 19.4834
Epoch 746/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.0147 - val_loss: 19.2489
Epoch 747/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.1507 - val_loss: 19.3588
Epoch 748/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.1209 - val_loss: 19.1416
Epoch 749/1000
24/24 [==============================] - 0s 4ms/step - loss: 19.0296 - val_loss: 19.2752
Epoch 750/1000
24/24 [==============================] - 0s 5ms/s

Epoch 820/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.7864 - val_loss: 19.1805
Epoch 821/1000
24/24 [==============================] - 0s 5ms/step - loss: 19.0122 - val_loss: 19.0736
Epoch 822/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.7353 - val_loss: 18.7501
Epoch 823/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.9806 - val_loss: 19.2775
Epoch 824/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.7572 - val_loss: 18.9747
Epoch 825/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.9737 - val_loss: 18.9081
Epoch 826/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.8759 - val_loss: 19.0365
Epoch 827/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.5377 - val_loss: 19.0780
Epoch 828/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.7367 - val_loss: 19.0869
Epoch 829/1000
24/24 [==============================] - 0s 5ms/s

Epoch 899/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.6409 - val_loss: 19.0501
Epoch 900/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.8116 - val_loss: 18.7470
Epoch 901/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.6541 - val_loss: 18.6165
Epoch 902/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.7010 - val_loss: 19.0633
Epoch 903/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.6458 - val_loss: 18.8042
Epoch 904/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.5618 - val_loss: 18.8189
Epoch 905/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.8500 - val_loss: 19.3695
Epoch 906/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.5544 - val_loss: 19.1397
Epoch 907/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.6461 - val_loss: 18.8600
Epoch 908/1000
24/24 [==============================] - 0s 5ms/s

Epoch 978/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.5870 - val_loss: 18.7674
Epoch 979/1000
24/24 [==============================] - 0s 4ms/step - loss: 18.4524 - val_loss: 18.7776
Epoch 980/1000
24/24 [==============================] - 0s 6ms/step - loss: 18.3304 - val_loss: 18.7598
Epoch 981/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.5807 - val_loss: 18.7756
Epoch 982/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.4960 - val_loss: 19.1759
Epoch 983/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.6370 - val_loss: 18.7406
Epoch 984/1000
24/24 [==============================] - 0s 6ms/step - loss: 18.6618 - val_loss: 18.7483
Epoch 985/1000
24/24 [==============================] - 0s 6ms/step - loss: 18.4815 - val_loss: 18.6848
Epoch 986/1000
24/24 [==============================] - 0s 5ms/step - loss: 18.3188 - val_loss: 18.8210
Epoch 987/1000
24/24 [==============================] - 0s 5ms/s

In [13]:
def latent_space_data(vae, data):
    encoder, decoder = vae
    z_mean, _, _ = encoder.predict(data)
    return z_mean
LS_Ger_train = latent_space_data(models, x_train)
LS_Ger_test = latent_space_data(models, x_test)

### MSPO

In [14]:
    #def SPOfn(latent_space_im,mino,majo,y_d):
       
    
    latent_space_im = LS_Ger_train
    mino = 1
    majo = 2
    y_d = y_train
    
    nTarget = np.sum(y_d == majo)


    posy = y_d == mino
    negy = y_d != mino
    P = latent_space_im[np.where(posy == True)[0],:]
    N = latent_space_im[np.where(negy == True)[0],:]

    #print(len(P),len(N))

    poscnt = P.shape[0]
    NumToGen = nTarget - poscnt
    Me  = np.mean((P),axis = 0)
    PCov = np.cov(P.T)
    #print(NumToGen)
    [D,V] = np.linalg.eig(PCov)
    #d = [D[x,x] for x in range(D.shape[0])]
    d = D
    n = P.shape[1] #Feature dimension
    idx = d.argsort()[::-1]   
    d = d[idx]
    V = V[:,idx]
    #d = d[0:n+1]
    #v = V[:,n::-1]
    
    Ind = (d<= 5e-03)

    if np.sum(Ind) != 0:
        M = (list(Ind).index(True)+1)
    else:
        M = n
        
    
    PN = np.concatenate((P,N),axis=0)
    TCov = np.cov(PN.T)
    dT    = np.dot(V.T,np.dot(TCov, V))
    dT = [dT[x,x] for x in range(dT.shape[0])]
    
    #Modify the Eigen spectrum according to a 1-Parameter Model
    dMod  = np.zeros((n,1))
    Alpha = d[0]* d[M-1]*(M-1) /(d[0] - d[M-1]) #d[0]* d[M-1]*(M-1) /(d[0] - d[M-1])
    Beta  = ((M)*d[M-1] - d[0])/(d[0] - d[M-1])
    
    for i in range(n):
        if i<M-1:

            dMod[i] = d[i]
        else:
            dMod[i] = Alpha/(i+1+Beta)
            if dMod[i] > dT[i]:
                dMod[i] = dT[i]

    R = 1.0
    d = dMod
            
    ########################################
   
    Rn = M
    Un = len(Me) - M
    Ptemp = P

    MuR = np.zeros((Rn,1)) #mlayer#
    SigmaR = np.identity((Rn)) #v_mat #

    MuU = np.zeros((Un,1))
    SigmaU = np.identity((Un))

    SampGen = np.zeros((int(NumToGen*R), len(Me)))
    SampSel = np.zeros((int(NumToGen*R), len(Me)))
    Prob    = np.zeros((int(NumToGen*R),1))

    cnt = 0
    DD = np.sqrt(d)
    MuR = MuR.reshape(MuR.shape[0],)
    MuU = MuU.reshape(MuU.shape[0],)
    
    while cnt < int(R*NumToGen):
        
        aR =  np.random.multivariate_normal(MuR.T, SigmaR, 1)
        tp = multivariate_normal.pdf(aR, MuR, SigmaR) #aR.pdf(1)
        #print(tp)

        if Un > 0:
            aU = np.random.multivariate_normal(MuU, SigmaU, 1)
            a = np.multiply(np.concatenate((aR,aU),axis=1).T,DD)   #The vector in Eigen transformed domain;
        else:
            a = np.multiply(aR.T,DD)
            #print(a)

        x = np.dot(a.T,V.T)+ Me
        #print(x)
        #pdb.set_trace()
        PDist = np.sqrt(np.sum(np.square((x-P)),axis=1))
        NDist = np.sqrt(np.sum(np.square((x-N)),axis=1))

        [tmp,ind]  = [np.min(NDist),np.argmin(NDist)]

        if np.min(PDist) < tmp:
            PPDist = np.sqrt(np.sum(np.square((N[ind,:]-P)),axis=1))
            if tmp >= np.min(PPDist) and tmp <= np.max(PPDist):
                SampGen[cnt,:] = x
                Prob[cnt,0] = tp  
                cnt+=1
                Ptemp = np.concatenate((Ptemp,SampGen),axis =0)

    for i in range (int(R*NumToGen)):
        [tmp,ind]  = [np.min(Prob),np.argmin(Prob)]
        Prob[ind] =  np.inf
        SampSel[i,:] = SampGen[ind,:]

    Ynew = SampSel #np.concatenate((SampSel,P),axis = 0)
        
    #return Ynew


In [15]:
Datanew = np.concatenate((SampSel,P),axis = 0)
Total = np.concatenate((Datanew,N),axis = 0)

In [16]:
label = np.zeros((Total.shape[0],))
label[0:Datanew.shape[0]] = 1
label[Datanew.shape[0]:Total.shape[0]] = 2

#### Metric function

In [17]:
def metric_score(y_test, y_pred):
    confMat=confusion_matrix(y_test, y_pred) 
    #print(confMat)
    TP = confMat[0,0]
    TN = confMat[1,1]
    FP = confMat[1,0]
    FN = confMat[0,1]
    #print(TP,FP,TN,FN)
    Eta = 1/2*((TP/np.sum(y_test==1))+(TN/np.sum(y_test==2)))
    TPR = TP/(TP+FN)
    TNR = TN/(TN+FP)
    Gmean = np.sqrt(TPR*TNR)
    return(TPR,TNR,Gmean,Eta)

In [18]:
X_train = (Total)
y_train = (label)
X_test = (LS_Ger_test)
y_test = (y_test)

mm_X = MinMaxScaler(feature_range=(0,1.6))
X_trainscaled= mm_X.fit_transform(X_train)#
X_testscaled= mm_X.transform(X_test) 

clf = MLPClassifier(solver='adam',hidden_layer_sizes=(10, 64, 9, ),activation="relu",
                    batch_size= 64,random_state=1,max_iter = 2000,learning_rate_init = 0.00012,
                   learning_rate= 'adaptive',power_t=0.6,warm_start = False,
                   beta_1 =0.95,beta_2 = 0.850)
# 10,32,8,3 5,10,8,3,
# (8,16,10,9,6,) -4 LS 10,16,12,6,3
# 10,25,10,8,6,(10,25,10,8,6,)
clf.fit(X_trainscaled, y_train)

y_pred=clf.predict(X_testscaled)

target_names = ['class 1', 'class 2']
print('TPR, TNR, GM , Eta:')
print(metric_score(y_test, y_pred))

TPR, TNR, GM , Eta:
(0.7790697674418605, 0.7242990654205608, 0.7511853995223504, 0.7516844164312106)


In [19]:
kf = KFold(n_splits=10,shuffle = True)
print("Cross Validation")
print('TPR, TNR, GM , Eta:')

for train_indices, test_indices in kf.split((X_trainscaled)):
    clf.fit(X_trainscaled[train_indices], y_train[train_indices])
    # print(clf.score(X_testscaled, y_test))

    y_pred=clf.predict(X_testscaled)
    print(metric_score(y_test, y_pred))
    

Cross Validation
TPR, TNR, GM , Eta:
(0.7441860465116279, 0.7429906542056075, 0.7435881101445605, 0.7435883503586177)
(0.7558139534883721, 0.719626168224299, 0.7374981350751307, 0.7377200608563356)
(0.7325581395348837, 0.7476635514018691, 0.7400723074220499, 0.7401108454683765)
(0.7674418604651163, 0.7242990654205608, 0.7455584633678972, 0.7458704629428385)
(0.7674418604651163, 0.7476635514018691, 0.7574881562703185, 0.7575527059334928)
(0.7558139534883721, 0.719626168224299, 0.7374981350751307, 0.7377200608563356)
(0.7674418604651163, 0.7102803738317757, 0.7383081277795309, 0.738861117148446)
(0.7674418604651163, 0.7149532710280374, 0.7407327915404972, 0.7411975657465768)
(0.7441860465116279, 0.719626168224299, 0.7318030835526403, 0.7319061073679635)
(0.7558139534883721, 0.7289719626168224, 0.7422716356210833, 0.7423929580525972)


In [20]:
print('Time taken :',time.time()-start_time)

Time taken : 234.11117315292358
